# PyTorch 모델 저장 및 불러오기

> [PyTorch 공식 튜토리얼: 모델 저장하기 & 불러오기](https://tutorials.pytorch.kr/beginner/saving_loading_models.html)

> [PyTorch 모델 저장하기](https://velog.io/@tjdtnsu/PyTorch-%EA%B8%B0%EC%B4%88-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B3%A0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0)

training이 끝난 model을 `torch.save`로 저장, `torch.load`로 불러올 수 있다.

정확히는 model states를 저장하고 불러오는 과정. model이 학습 중 차지하는 memory 대부분에 해당한다. model states는 다음을 포함한다.

- optimizer states: 예를 들어 Adam에서의 momentum과 variance 등

- gradients

- parameters

---

## 1.1 torch.save

model 인스턴스 자체를 parameter로 `torch.save`에 넘기면 `.pt`(혹은 `.pth`) 확장자로 저장할 수 있다. 둘의 차이는 없지만 주로 `.pt`를 사용하기를 권장한다.

이때 Python의 내장된, data를 저장할 때 사용하는 library인 'Pickle'을 이용한다. (graph 구조와 weights들이 담긴) Python object가 byte stream으로 **serializing**(직렬화)된다. 

- models, tensors, dictionaries 등 모든 object들이 serialization된다.

> [pickle 모듈 설명](https://docs.python.org/ko/3/library/pickle.html)

> serialization(직렬화)란 현재 data(object나 structure)를 저장하거나 다른 환경으로 전달하기 위해 byte 형태의 '정해진 포맷'으로 변환하는 과정을 의미한다.

---

### 1.1.1 state_dict

PyTorch에서 모든 neural network의 base class에 해당되는 module인 `torch.nn.Module`를 잠시 살펴보자.

Pytorch에서 사용자 정의로 model을 구성할 때는, 기본적으로 다음과 같이 `torch.nn.Module`을 상속받는 class를 만들어야 한다.


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Custom_Model(nn.Module):
    def __init__(self):
        super(Custom_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)
        """
        위 예시처럼 구성하면 된다.
        self.module1 = ...
        self.module2 = ...
        """
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        """
        위 예시처럼 구성하면 된다.
        x = function1(x)
        x = function2(x)
        """
        return x

# 참고로 nn.Module 상속 인스턴스에 패러미터로 데이터를 전달해 주면, forward() 메서드를 바로 수행하게 된다.
model = Custom_Model()

`torch.nn.Module`의 인스턴스(위 예시의 'model')에서 **learable parameters**(weights, biases)에 접근하는 방법이 있다.

- 위 예시를 기준으로 `model.parameters()`로 접근할 수 있다.

> 특히 learnable parameters를 (loss를 계산하고 역전파 등의 방법으로) optimize(최적화)하는 `torch.optim`의 optimizer 객체가 `model.paramters()`를 인자로 받는다.

또한 이러한 learnable parameter들은 `state_dict`라는 **Python 딕셔너리 객체로, 각 layer와 parameter tensor가 서로 mapping된 구조**다. 이뿐만이 아니라 registered buffers(batch normalization의 running mean 같은 요소)도 `state_dict`에 포함되어 있다.

`torch.save` model 전체 정보가 아니라, weight, bias 등의 parameter가 담긴 Python 딕셔너리 `state_dict`만을 저장하는 수단도 제공하고 있다.

In [ ]:
# model 전체 정보를 현재 경로에 저장
torch.save(model, './model.pt')

# model의 state_dict() 정보만 현재 경로에 저장
torch.save(model.state_dict(), './model.pt')

위 두 가지 방법으로 저장한 model을 불러오는 방법도 각각 다르다. 코드를 보자.

In [ ]:
# 전체 정보를 저장한 파일을 불러온다.
model = torch.load('./model.pt')

# state_dict() 정보만 저장한 파일을 불러온다.
model.load_state_dict(torch.load('./model.pt'))

> 이외 여러 model을 한 파일에 저장하는 방법도 있다.

---

## 1.2 checkpoint 저장 및 불러오기

training 혹은 inference 중 model checkpoint를 `torch.save()`를 통해 저장할 수 있다. 하지만 checkpoint 저장에는 `state_dict`만이 아닌 다른 정보들도 꼭 담아야 한다.(이러한 이유 때문에 종종 model의 자체적인 크기보다 훨씬 큰 size를 갖게 된다.) 아래는 checkpoint 저장 시 담을 수 있는 정보들의 예시다.

- `state_dict`를 저장할 수 있다.

- optimizer의 `state_dict`를 저장할 수 있다.

- 저장 전까지 몇 epoch을 training했는지 등의 정보도 담을 수 있다.

- training loss나 cost 정보도 담을 수 있다.

    - loss: training data를 이용해 계산되는 성능 측정 지표

    - cost: validation(혹은 test) data를 이용해, model이 얼마나 새로운 data를 잘 예측하는지 나타내는 지표(즉, generalization이 얼마나 잘 됐는지를 파악하는 지표)

- checkpoint 설명에 해당되는 description도 담을 수 있다.

PyTorch에서 checkpoint를 저장할 때는 `.tar` 확장자를 사용하는 것이 일반적이다.(하지만 `.pt` 확장자로도 저장할 수 있다.)

In [ ]:
# save checkpoint
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch
}, "./checkpoint.tar")


이제 이렇게 저장한 checkpoint를 불러오고 사용하는 방법을 파악해 보자. 앞서 checkpoint는 여러가지 정보를 담았으니까, 불러온 checkpoint에서 `state_dict`나 optimizer의 `state_dict`, epoch, loss 등을 분리해야 한다.

또한 checkpoint를 불러온 뒤, 다음과 같이 mode를 설정해 줘야 한다.

- training을 지속하기 위해서는 `model.train()`을 호출해서 **training mode로 설정해야 한다**.

- inference를 실행하기 위해서는 반드시 `model.eval()`을 호출해서 **evaluation mode로 바꿔줘야 한다**. 

> 이는 dropout이나 batch normalization과 같은 layer들이 training과 inference 시의 동작이 다르기 때문이다.

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)           # checkpoint 파일을 불러온다.
model.load_state_dict(checkpoint['model_state_dict'])         # checkpoint에서 model의 state_dict를 불러온다.
optimizer.load_state_dict(checkpoint['optimizer_state_dict']) # optimizer의 state_dict를 불러온다.
epoch = checkpoint['epoch']             # epoch 정보를 불러온다.
loss = checkpoint['loss']               # loss 정보를 불러온다.

"""
# inference mode
model.eval()

# training mode
model.train()
"""

---

## 1.3 CPU, GPU 선택하여 불러오기

training은 GPU로 하고, inference는 CPU로 하는 등 작업을 수행하는 장치를 다르게 선택할 수도 있다. 이때는 `torch.load`의 `map_location` 인자에 정보를 전달해 주면 된다.

- CPU: "cpu"

- GPU: "cuda"

In [ ]:
# GPU를 우선하되 없으면 CPU로 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("./model_state_dict.pth", map_location=device)

# 공식 튜토리얼 코드 버전(GPU에 모델 불러오기)
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # 사용할 GPU 장치 번호를 선택해야 한다.
model.to(device)

---

## 1.4 다른 구조 모델의 parameter 불러오기

종종 내가 만든 정형화되지 않은 구조의 custom model에 pre-trained model을 가져오려고 하면, 두 model의 `state_dict`가 달라서 에러가 발생하게 된다. 이 경우 `load_state_dict()` 메서드에 `strict` 인수로 'False'를 전달하면 사용할 수 있는 것만 불러오게 된다.


In [ ]:
model.load_state_dict(torch.load('./model_state_dict.pt'), strict=False)